In [6]:
import numpy as np
import pandas as pd
import random
import spacy
import translators as ts

nlp = spacy.load('en_core_web_sm')

#round 1
#balance label

#round2

#random swap
def random_swap(sentence, swap_num = 2, nlp = spacy.load('en_core_web_sm')):
    tokens = nlp(sentence)
    str_list = [str(tok) for tok in tokens]
    for i in range (1,swap_num):
        a,b = random.randint(0,len(str_list)-1),random.randint(0,len(str_list)-1)
        temp = str_list[a]
        str_list[a] = str_list[b]
        str_list[b] = temp
    aug_sentence = ' '.join(str_list)    
    
    return aug_sentence

#random delete
def random_delete(sentence, delete_num = 1, nlp = spacy.load('en_core_web_sm')):
    tokens = nlp(sentence)
    str_list = [str(tok) for tok in tokens]
    if len(str_list) <= 5:
        return ' '.join(str_list)
    if delete_num < 0:
        delete_num = 0
    for i in range(delete_num):
        a = random.randint(0,len(str_list)-1)
        str_list.remove(str_list[a])
    aug_sentence = ' '.join(str_list)
    return aug_sentence

#translate and translate
def double_translate(sentence,lang = 'zh-TW'):
    #https://www.loc.gov/standards/iso639-2/php/code_list.php
    first_trans = ts.google(sentence, from_language='en', to_language= lang)
    return ts.google(first_trans, from_language=lang ,to_language='en')
    
def label_count(data):
    labeldict = {}
    df = pd.read_csv(data,sep=',')

    for index,row in df.iterrows():
        emotion = row['Emotion']
        if emotion in labeldict:
            labeldict[emotion] += 1
        else:
            labeldict[emotion] = 1

    print(labeldict)

'neutral': 5960, 'surprise': 1490, 'fear': 338, 'sadness': 876, 'joy': 2312, 'disgust': 364, 'anger': 1500

In [3]:
import numpy as np
import pandas as pd

predict = []
labeldict = {}

data = "trainset_AUGr1.csv"
label_count(data)


{'neutral': 5960, 'surprise': 1490, 'joy': 2312, 'sadness': 876, 'fear': 676, 'disgust': 728, 'anger': 1500}


In [63]:
#round 1
#先做一次train 一次val，之後再合起來

data = "merged.csv"
df = pd.read_csv(data,sep=',')
df.drop(df[(df.Emotion != 'fear')& (df.Emotion != 'disgust')].index, inplace=True)
nlp = spacy.load('en_core_web_sm')

for index,row in df.iterrows():
    delnum = random.randint(1,3)
    swap = random.randint(1,5)
    new_sentence = double_translate(row['Utterance'])
    new_sentence = random_delete(new_sentence,swap,nlp)
    new_sentence = random_swap(new_sentence,delnum,nlp)
    row['Utterance'] = new_sentence
    
df2 = pd.read_csv(data,sep=',')
df3 = pd.concat([df,df2])
df3 = df3.sample(frac=1)
df3.to_csv('merged_ay.csv',index = False)

{'neutral': 12520, 'surprise': 3190, 'fear': 1122, 'sadness': 1897, 'joy': 4838, 'disgust': 1154, 'anger': 3208}


"\ndf.drop(df[(df.Emotion != 'fear')& (df.Emotion != 'disgust')].index, inplace=True)\n\nnlp = spacy.load('en_core_web_sm')\n\nfor index,row in df.iterrows():\n    delnum = random.randint(1,3)\n    swap = random.randint(1,5)\n    new_sentence = double_translate(row['Utterance'])\n    new_sentence = random_delete(new_sentence,swap,nlp)\n    new_sentence = random_swap(new_sentence,delnum,nlp)\n    row['Utterance'] = new_sentence\n    \ndf2 = pd.read_csv(data,sep=',')\ndf3 = pd.concat([df,df2])\ndf3 = df3.sample(frac=1)\ndf3.to_csv('merged_aug_temp.csv',index = False)\n"

In [2]:
#round1.5
from tqdm import tqdm
data = "trainset_AUGr1.csv"
df = pd.read_csv(data,sep=',')
df.drop(df[(df.Emotion != 'fear')& (df.Emotion != 'disgust')&(df.Emotion !='sadness')].index, inplace=True)
nlp = spacy.load('en_core_web_sm')

for index,row in tqdm(df.iterrows()):
    delnum = random.randint(1,3)
    swap = random.randint(1,5)
    new_sentence = random_delete(row['Utterance'],delnum,nlp)
    new_sentence = random_swap(new_sentence,swap,nlp)
    new_sentence = double_translate(new_sentence,lang = 'ko')#korean
    new_sentence = random_swap(new_sentence,swap,nlp)
    row['Utterance'] = new_sentence
    
df2 = pd.read_csv(data,sep=',')
df3 = pd.concat([df,df2])
df3 = df3.sample(frac=1)
df3.to_csv('trainset_AUGr1p5.csv',index = False)


2280it [1:46:28,  2.80s/it]


NameError: name 'labeldict' is not defined

In [7]:
label_count('trainset_AUGr1p5.csv')

{'joy': 2312, 'anger': 1500, 'neutral': 5960, 'sadness': 1752, 'fear': 1352, 'surprise': 1490, 'disgust': 1456}


In [8]:
#round 1.75
data = "trainset_AUGr1p5.csv"
df = pd.read_csv(data,sep=',')
df.drop(df[df.Emotion == 'neutral'].index, inplace=True)
nlp = spacy.load('en_core_web_sm')

for index,row in tqdm(df.iterrows()):
    delnum = random.randint(1,3)
    swap = random.randint(2,5)
    new_sentence = random_delete(row['Utterance'],delnum,nlp)
    new_sentence = random_swap(new_sentence,swap,nlp)
    new_sentence = double_translate(new_sentence,lang = 'tr')#turkish
    new_sentence = random_swap(new_sentence,swap,nlp)
    row['Utterance'] = new_sentence
    
df2 = pd.read_csv(data,sep=',')
df3 = pd.concat([df,df2])
df3 = df3.sample(frac=1)
df3.to_csv('trainset_AUGr1p75.csv',index = False)
label_count('trainset_AUGr1p75.csv')

9862it [9:05:21,  3.32s/it]


{'disgust': 2912, 'anger': 3000, 'joy': 4624, 'surprise': 2980, 'fear': 2704, 'neutral': 5960, 'sadness': 3504}


In [80]:
#round 2
data = "trainset_Augr1.csv"
df = pd.read_csv(data,sep=',')
df.drop(df[df.Utterance.apply(lambda x: len(x.split(' ')) < 6)].index,inplace = True)

nlp = spacy.load('en_core_web_sm')

for index,row in df.iterrows():
    delnum = random.randint(1,3)
    swap = random.randint(3,7)
    new_sentence = random_delete(new_sentence,swap,nlp)
    new_sentence = random_swap(new_sentence,delnum,nlp)
    row['Utterance'] = new_sentence

df2 = pd.read_csv(data,sep=',')
df3 = [df,df2]
result = pd.concat(df3)
result.to_csv('train_Aug_round2.csv',index = False)

In [50]:
#merge train valid

data = "trainset_AUGr1.csv"
df = pd.read_csv(data)
data2 = "valid_augset.csv"
df2 = pd.read_csv(data2)
print(len(df))
print(len(df2))
df3 = [df,df2]
df3 = pd.concat(df3)
print(len(df3))
df3.to_csv('merged_Aug_r1.csv',index = False)

13542
1547
15089


In [52]:
result

,Utterance,Speaker,Emotion,Dialogue_ID,Utterance_ID,Old_Dialogue_ID,Old_Utterance_ID,Season,Episode,StartTime,EndTime
0,also I was the point person on my company’s tr...,Chandler,neutral,0,0,0,0,8,21,"00:16:16,059","00:16:21,731"
1,You must’ve had your hands full.,The Interviewer,neutral,0,1,0,1,8,21,"00:16:21,940","00:16:23,442"
2,That I did. That I did.,Chandler,neutral,0,2,0,2,8,21,"00:16:23,442","00:16:26,389"
3,So let’s talk a little bit about your duties.,The Interviewer,neutral,0,3,0,3,8,21,"00:16:26,820","00:16:29,572"
4,My duties? All right.,Chandler,surprise,0,4,0,4,8,21,"00:16:34,452","00:16:40,917"
...,...,...,...,...,...,...,...,...,...,...,...
12835,"When I first moved to the city, I went out a c...",Joey,disgust,2159,2,1038,11,2,3,"00:00:35,744","00:00:44,334"
12836,It made me nuts.,Joey,disgust,2159,3,1038,12,2,3,"00:00:45,462","00:00:46,587"
12837,"You guys are messing with me, right?",Joey,surprise,2159,4,1038,15,2,3,"00:01:00,518","00:01:03,520"
12838,Yeah.,All,neutral,2159,5,1038,16,2,3,"00:01:05,398","00:01:07,274"


In [9]:
from tqdm import tqdm
data = "validset_AUGr1.csv"
df = pd.read_csv(data,sep=',')
df.drop(df[(df.Emotion != 'fear')& (df.Emotion != 'disgust')&(df.Emotion !='sadness')].index, inplace=True)
nlp = spacy.load('en_core_web_sm')

for index,row in tqdm(df.iterrows()):
    delnum = random.randint(1,3)
    swap = random.randint(1,5)
    new_sentence = random_delete(row['Utterance'],delnum,nlp)
    new_sentence = random_swap(new_sentence,swap,nlp)
    new_sentence = double_translate(new_sentence,lang = 'ko')#korean
    new_sentence = random_swap(new_sentence,swap,nlp)
    row['Utterance'] = new_sentence
    
df2 = pd.read_csv(data,sep=',')
df3 = pd.concat([df,df2])
df3 = df3.sample(frac=1)
df3.to_csv('validset_AUGr1p5.csv',index = False)
label_count('validset_AUGr1p5.csv')

#round 1.75
data = "validset_AUGr1p5.csv"
df = pd.read_csv(data,sep=',')
df.drop(df[df.Emotion == 'neutral'].index, inplace=True)
nlp = spacy.load('en_core_web_sm')

for index,row in tqdm(df.iterrows()):
    delnum = random.randint(1,3)
    swap = random.randint(2,5)
    new_sentence = random_delete(row['Utterance'],delnum,nlp)
    new_sentence = random_swap(new_sentence,swap,nlp)
    new_sentence = double_translate(new_sentence,lang = 'tr')#turkish
    new_sentence = random_swap(new_sentence,swap,nlp)
    row['Utterance'] = new_sentence
    
df2 = pd.read_csv(data,sep=',')
df3 = pd.concat([df,df2])
df3 = df3.sample(frac=1)
df3.to_csv('validset_AUGr1p75.csv',index = False)
label_count('validset_AUGr1p75.csv')

315it [16:12,  3.09s/it]


{'neutral': 600, 'sadness': 290, 'disgust': 124, 'surprise': 210, 'joy': 214, 'anger': 208, 'fear': 216}


1262it [1:09:54,  3.32s/it]


{'anger': 416, 'disgust': 248, 'surprise': 420, 'joy': 428, 'neutral': 600, 'sadness': 580, 'fear': 432}


In [10]:
data = "trainset_AUGr1p75.csv"
df = pd.read_csv(data)
data2 = "validset_AUGr1p75.csv"
df2 = pd.read_csv(data2)
print(len(df))
print(len(df2))
df3 = [df,df2]
df3 = pd.concat(df3)
print(len(df3))
df3.to_csv('mergedset_Augr1p75.csv',index = False)

25684
3124
28808
